Parallelism consists of performing multiple operations at the same time. 

Multiprocessing is a means to effect parallelism, and it entails spreading tasks over a computer’s central processing units (CPUs, or cores). Multiprocessing is well-suited for CPU-bound tasks: tightly bound for loops and mathematical computations usually fall into this category.

Concurrency is a slightly broader term than parallelism. It suggests that multiple tasks have the ability to run in an overlapping manner. (There’s a saying that concurrency does not imply parallelism.)

Threading is a concurrent execution model whereby multiple threads take turns executing tasks. One process can contain multiple threads. Python has a complicated relationship with threading thanks to its GIL, but that’s beyond the scope of this article.

**async IO** is not a newly invented concept, and it has existed or is being built into other languages and runtime environments, such as Go, C#, or Scala.

In fact, async IO is a single-threaded, single-process design: it uses **cooperative multitasking**, a term that you’ll flesh out by the end of this tutorial.

**What is Asynchronous?**

- Asynchronous routines are able to “pause” while waiting on their ultimate result and let other routines run in the meantime.

- Asynchronous code, through the mechanism above, facilitates concurrent execution. To put it differently, asynchronous code gives the look and feel of concurrency.


![Alt text](image.png)

cooperative multitasking is a fancy way of saying that a program’s event loop (more on that later) communicates with multiple tasks to let each take turns running at the optimal time.

There are two keywords, async and await, serve different purposes but come together to help you declare, build, execute, and manage asynchronous code.

**a coroutine** is a function that can **suspend its execution** before reaching return, and it can **indirectly pass control** to another coroutine for some time.